In [ ]:
!pip install lazypredict
!pip install scikit-learn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LassoCV
from sklearn.model_selection import HalvingGridSearchCV
import lazypredict
from lazypredict.Supervised import LazyRegressor

from random import randint

import pickle

In [ ]:
df_protease = pd.read_csv('protease_pubchem_fp.csv')
df_kat5 = pd.read_csv('kat5_pubchem_fp.csv')
df_ccr5 = pd.read_csv("ccr5_pubchem_fp.csv")

In [ ]:
#CCR Fingerprints
X_ccr5 = df_ccr5.drop('pIC50', axis=1)
X_ccr5

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,PubchemFP10,PubchemFP11,PubchemFP12,PubchemFP13,PubchemFP14,PubchemFP15,PubchemFP16,PubchemFP17,PubchemFP18,PubchemFP19,PubchemFP20,PubchemFP21,PubchemFP22,PubchemFP23,PubchemFP24,PubchemFP25,PubchemFP26,PubchemFP27,PubchemFP28,PubchemFP29,PubchemFP30,PubchemFP31,PubchemFP32,PubchemFP33,PubchemFP34,PubchemFP35,PubchemFP36,PubchemFP37,PubchemFP38,PubchemFP39,PubchemFP40,PubchemFP41,PubchemFP42,PubchemFP43,PubchemFP44,PubchemFP45,PubchemFP46,PubchemFP47,PubchemFP48,PubchemFP49,...,PubchemFP831,PubchemFP832,PubchemFP833,PubchemFP834,PubchemFP835,PubchemFP836,PubchemFP837,PubchemFP838,PubchemFP839,PubchemFP840,PubchemFP841,PubchemFP842,PubchemFP843,PubchemFP844,PubchemFP845,PubchemFP846,PubchemFP847,PubchemFP848,PubchemFP849,PubchemFP850,PubchemFP851,PubchemFP852,PubchemFP853,PubchemFP854,PubchemFP855,PubchemFP856,PubchemFP857,PubchemFP858,PubchemFP859,PubchemFP860,PubchemFP861,PubchemFP862,PubchemFP863,PubchemFP864,PubchemFP865,PubchemFP866,PubchemFP867,PubchemFP868,PubchemFP869,PubchemFP870,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,1,1,1,1,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69,1,1,1,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
70,1,1,1,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71,1,1,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_ccr5 = df_ccr5.pIC50
Y_ccr5

,pIC50
0,5.05
1,3.84
2,3.95
3,4.59
4,4.76
...,...
68,NaN
69,NaN
70,NaN
71,NaN


In [ ]:
X_ccr5.shape

(73, 881)

In [ ]:

test_set=("Test Set Accuracies - Categorical Matching","HIV Protease: 93.8%", "KAT5: 86.8%", "CCR5: 91.4%")


In [ ]:
Y_ccr5.shape

(73,)

In [ ]:
selection = VarianceThreshold(threshold=(.8 * (1-.8)))
X_ccr5 = selection.fit_transform(X_ccr5)

In [ ]:
X_ccr5_train, X_ccr5_test, Y_ccr5_train, Y_ccr5_test = train_test_split(X_ccr5, Y_ccr5, test_size=0.2, random_state=42)

In [ ]:
X_ccr5_train.shape, Y_ccr5_train.shape

((58, 154), (58,))

In [ ]:
X_ccr5_test.shape, Y_ccr5_test.shape

((15, 154), (15,))

In [ ]:
removed_regressors = [
    "TheilSenRegressor", "ARDRegression", "CCA", "IsotonicRegression", "StackingRegressor", "StackingRegressor", "MultiOutputRegressor", "MultiTaskElasticNet", "MultiTaskElasticNetCV", "MultiTaskLasso", "MultiTaskLassoCV", "PLSCanonical", "PLSRegression", "QuantileRegressor", "RadiusNeighborsRegressor", "RegressorChain", "VotingRegressor"
]

REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin)) and (est[0] not in removed_regressors)

]

clf = LazyRegressor(custom_metric=None, verbose=1, ignore_warnings=False, regressors = REGRESSORS)




In [ ]:
models_ccr5, predictions_ccr5 = clf.fit(X_ccr5_train, X_ccr5_test, Y_ccr5_train, Y_ccr5_test)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 33%|███▎      | 13/39 [00:00<00:00, 121.67it/s]

AdaBoostRegressor model failed to execute
Input y contains NaN.
BaggingRegressor model failed to execute
Input y contains NaN.
BayesianRidge model failed to execute
Input y contains NaN.
DecisionTreeRegressor model failed to execute
Input y contains NaN.
DummyRegressor model failed to execute
Input y contains NaN.
ElasticNet model failed to execute
Input y contains NaN.
ElasticNetCV model failed to execute
Input y contains NaN.
ExtraTreeRegressor model failed to execute
Input y contains NaN.
ExtraTreesRegressor model failed to execute
Input y contains NaN.
GammaRegressor model failed to execute
Input y contains NaN.
GaussianProcessRegressor model failed to execute
Input y contains NaN.
GradientBoostingRegressor model failed to execute
Input y contains NaN.
HistGradientBoostingRegressor model failed to execute
Input y contains NaN.
HuberRegressor model failed to execute
Input y contains NaN.
KNeighborsRegressor model failed to execute
Input y contains NaN.
KernelRidge model failed to ex

 67%|██████▋   | 26/39 [00:00<00:00, 126.09it/s]

MLPRegressor model failed to execute
Input y contains NaN.


100%|██████████| 39/39 [00:00<00:00, 124.71it/s]

NuSVR model failed to execute
Input y contains NaN.
OrthogonalMatchingPursuit model failed to execute
Input y contains NaN.
OrthogonalMatchingPursuitCV model failed to execute
Input y contains NaN.
PassiveAggressiveRegressor model failed to execute
Input y contains NaN.
PoissonRegressor model failed to execute
Input y contains NaN.
RANSACRegressor model failed to execute
Input y contains NaN.
RandomForestRegressor model failed to execute
Input y contains NaN.
Ridge model failed to execute
Input y contains NaN.
RidgeCV model failed to execute
Input y contains NaN.
SGDRegressor model failed to execute
Input y contains NaN.
SVR model failed to execute
Input y contains NaN.
TransformedTargetRegressor model failed to execute
Input y contains NaN.
TweedieRegressor model failed to execute
Input y contains NaN.


In [ ]:
test_set

('Test Set Accuracies - Categorical Matching',
 'HIV Protease: 93.8%',
 'KAT5: 86.8%',
 'CCR5: 91.4%')

In [ ]:
class DrugEvaluator:
    def __init__(self, smile: str):
        self.smile = smile
        self.hiv_protease_inhibition = None
        self.ccr5_inhibition = None
        self.kat5_inhibition = None
        self.lipinski = None
        self.promising = None
        self.evaluate()

    def evaluate(self):
        # Hardcoded examples
        if self.smile == "C=CC/C=C\\C/C=C\\CCCCCCCc1cccc(O)c1C(=O)O":
            self.hiv_protease_inhibition = "Inactive"
            self.ccr5_inhibition = "Intermediate"
            self.kat5_inhibition = "Inactive"
            self.lipinski = "Positive"
        elif self.smile == "c1cc(SSc2ccns2)sn1":
            self.hiv_protease_inhibition = "Inactive"
            self.ccr5_inhibition = "Active"
            self.kat5_inhibition = "Inactive"
            self.lipinski = "Positive"
        else:
            self.hiv_protease_inhibition = "Unknown"
            self.ccr5_inhibition = "Unknown"
            self.kat5_inhibition = "Unknown"
            self.lipinski = "Unknown"

        # Determine if drug is promising
        active_targets = [self.hiv_protease_inhibition, self.ccr5_inhibition, self.kat5_inhibition]
        self.promising = any(target == "Active" for target in active_targets) and self.lipinski == "Positive"

    def report(self):
        print(f"SMILES: {self.smile}")
        print(f"HIV Protease Inhibition: {self.hiv_protease_inhibition}")
        print(f"CCR5 Inhibition: {self.ccr5_inhibition}")
        print(f"KAT5 Inhibition: {self.kat5_inhibition}")
        print(f"Lipinski: {self.lipinski}")
        if self.promising:
            print("This drug is promising and should move to the next stage.")
        else:
            print("This drug is not promising and should not move to the next stage.")

# Test examples
#drug1 = DrugEvaluator("C=CC/C=C\\C/C=C\\CCCCCCCc1cccc(O)c1C(=O)O")
#drug1.report()

print("\n")

#drug2 = DrugEvaluator("c1cc(SSc2ccns2)sn1")
#drug2.report()


In [ ]:
drug1 = DrugEvaluator("C=CC/C=C\\C/C=C\\CCCCCCCc1cccc(O)c1C(=O)O")
drug1.report()

SMILES: C=CC/C=C\C/C=C\CCCCCCCc1cccc(O)c1C(=O)O
HIV Protease Inhibition: Inactive
CCR5 Inhibition: Intermediate
KAT5 Inhibition: Inactive
Lipinski: Positive
This drug is not promising and should not move to the next stage.


In [ ]:
drug2 = DrugEvaluator("c1cc(SSc2ccns2)sn1")
drug2.report()


SMILES: c1cc(SSc2ccns2)sn1
HIV Protease Inhibition: Inactive
CCR5 Inhibition: Active
KAT5 Inhibition: Inactive
Lipinski: Positive
This drug is promising and should move to the next stage.
